#Installation



In [ ]:
!pip install tensorflow tensorflow_hub pandas

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd


#Load USE model and the dataset



In [ ]:
# Load USE model
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

#emotion df
data = pd.read_csv("isear.csv")
data.columns = ['Emotions','Text']
data.head()

#restarant df
#restaurantDF = pd.read_csv('/content/Restaurant_Reviews.tsv', sep='\t')
#restaurantDF.head()

,Emotions,Text
0,fear,Every time I imagine that someone I love or I ...
1,anger,When I had been obviously unjustly treated and...
2,sadness,When I think about the short time that we live...
3,disgust,At a gathering I found myself involuntarily si...
4,shame,When I realized that I was directing the feeli...


#Sentence Encoder

In [ ]:
#func that take column and encode it using USE and return it
def sentences_encoder(col):
    sentences = col.tolist()

    # Create USE vectors for sentences
    sentence_vectors = use_model(sentences)

    return sentence_vectors

In [ ]:

sentence_vectors = sentences_encoder(data['Text'])

sentence_vectors

<tf.Tensor: shape=(7651, 512), dtype=float32, numpy=
array([[ 7.2273105e-02,  2.6248939e-02, -5.1479443e-04, ...,
        -3.9996561e-02, -9.1328705e-04,  4.3221866e-03],
       [ 1.7619750e-02,  3.4595881e-02,  5.0737157e-02, ...,
         7.2480440e-02,  5.0265424e-02,  1.5052965e-02],
       [ 5.6326624e-02, -4.9449559e-02, -8.4469402e-03, ...,
         8.6149841e-05,  1.3908443e-02,  1.3678342e-02],
       ...,
       [ 3.5576940e-02,  7.5164363e-02, -6.2965922e-02, ...,
        -2.0096764e-02,  3.5482198e-02, -3.3691004e-02],
       [ 2.0432381e-02,  3.3611961e-02,  4.9654849e-02, ...,
        -5.6750245e-02, -2.5160605e-02, -2.2401934e-02],
       [ 5.8521178e-02,  6.5617055e-02, -1.1254602e-02, ...,
         2.2619285e-02,  3.4587462e-02, -5.6409170e-03]], dtype=float32)>

#Train and Create the model (Logistic)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Convert TensorFlow tensor to NumPy array
sentence_vectors = sentence_vectors.numpy()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sentence_vectors, data['Emotions'], test_size=0.2, random_state=42)

# Train a simple Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
# Predict emotions for test data
predictions = model.predict(X_test)

# Evaluate accuracy
accuracy = (predictions == y_test).mean()
print("Accuracy:", accuracy)

print("Classification Report:\n", classification_report(y_test, predictions))

Accuracy: 0.6074461136512084
Classification Report:
               precision    recall  f1-score   support

       anger       0.47      0.54      0.50       208
     disgust       0.62      0.54      0.58       226
        fear       0.67      0.73      0.70       208
       guilt       0.55      0.51      0.53       221
         joy       0.72      0.76      0.74       224
     sadness       0.72      0.65      0.68       222
       shame       0.52      0.52      0.52       222

    accuracy                           0.61      1531
   macro avg       0.61      0.61      0.61      1531
weighted avg       0.61      0.61      0.61      1531



In [ ]:
text_vec = use_model(["At a gathering I found myself involuntarily sitting next to two people who expressed opinions that I considered very low and discriminating.",])
text_vec = text_vec.numpy()

predict = model.predict(text_vec)

predict

array(['shame'], dtype=object)

#SVM

In [ ]:
from sklearn.svm import SVC

# Train a Linear SVM classifier
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
from sklearn.metrics import classification_report


# Make predictions on the test data
y_pred = svm_model.predict(X_test)
# Evaluate accuracy
accuracy = (y_pred == y_test).mean()
print("Accuracy:", accuracy)

# Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6048334421946441
Classification Report:
               precision    recall  f1-score   support

       anger       0.46      0.55      0.50       208
     disgust       0.60      0.55      0.58       226
        fear       0.68      0.67      0.68       208
       guilt       0.55      0.51      0.53       221
         joy       0.72      0.78      0.75       224
     sadness       0.74      0.64      0.69       222
       shame       0.52      0.53      0.52       222

    accuracy                           0.60      1531
   macro avg       0.61      0.60      0.61      1531
weighted avg       0.61      0.60      0.61      1531



#RandomForst

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Train a Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:


# Make predictions on the test data
yrf_pred = rf_model.predict(X_test)

# Evaluate accuracy
accuracy = (yrf_pred == y_test).mean()

print(accuracy)
# Evaluate the model
print("Classification Report:\n", classification_report(y_test, yrf_pred))

0.5519268451992162
Classification Report:
               precision    recall  f1-score   support

       anger       0.38      0.47      0.42       208
     disgust       0.50      0.46      0.47       226
        fear       0.61      0.73      0.67       208
       guilt       0.52      0.43      0.47       221
         joy       0.64      0.81      0.72       224
     sadness       0.74      0.57      0.64       222
       shame       0.50      0.41      0.45       222

    accuracy                           0.55      1531
   macro avg       0.55      0.55      0.55      1531
weighted avg       0.56      0.55      0.55      1531



#What is USE

(USE) The Universal Sentence Encoder  is model that encodes text into high-dimensional vectors that can be used for a variety of natural language processing (NLP) tasks. It was developed by Google AI and is available as a pre-trained model on the TensorFlow Hub.


The output of the model is a 512-dimensional vector that represents the meaning of the input text.

there two  main verision for USE one is Transformers Based and the other DAN based
we are using the DAN based version ,
 The model is a deep learning model that uses a technique called deep averaging network (DAN).




The USE model can be used for a variety of NLP tasks, including:

Text classification: Categorizing text into different classes, such as news, social media, or product reviews.

Semantic similarity: Measuring the similarity between two pieces of text.

Clustering: Grouping similar pieces of text together.

Question answering: Answering questions about text.

Machine translation: Translating text from one language to another.


Unlike traditional word embeddings like Word2Vec or GloVe that generate vectors for individual words, the Universal Sentence Encoder focuses on entire sentences or short text snippets.


 This makes it particularly useful for scenarios where understanding the meaning of a full sentence in context is important.